In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import time
import re

In [14]:
# 수동
page = 1
site = 'https://steamcommunity.com/app/753420/homecontent/?'

for idx in range(round(10 / 10)) :
    time.sleep(1)
    page = idx + 1
    param = {
                'userreviewsoffset': 10 * (page - 1),
                'p': page,
                'workshopitemspage': page,
                'readytouseitemspage': page,
                'mtxitemspage': page,
                'itemspage': page,
                'screenshotspage': page,
                'videospage': page,
                'artpage': page,
                'allguidepage': page,
                'webguidepage': page,
                'integratedguidepage': page,
                'discussionspage': page,
                'numperpage': 10,
                'browsefilter': 'toprated',
                'appHubSubSection': 10,
                'l': 'english',
                'filterLanguage': 'default',
                'searchText': '',
                'forceanon': 1
            }

    response = requests.get(site,params=param)
    bs = BeautifulSoup(response.content, 'html.parser')
    div_list = bs.select('.apphub_CardContentMain')
    for apphub_UserReviewCardContent in div_list :
        helpful = apphub_UserReviewCardContent.select('.found_helpful')[0].text
        helpful = helpful.strip()
        title = apphub_UserReviewCardContent.select('.title')[0].text
        date = apphub_UserReviewCardContent.select('.date_posted')[0].text
        date= date.strip()
        date = date.replace('Posted: ','')
        reviews = apphub_UserReviewCardContent.select('.apphub_CardTextContent')[0].text
        reviews = reviews.strip() # 좌우 공백 제거
        reviews = reviews.replace('Product received for free\r\n', '') # 무료 문자 제거
        reviews = reviews.replace('Early Access Review\r\n', '') # 얼리억세스 문자 제거
        reviews = re.sub('[\r\t]','',reviews) # 특수문자 제거
        reviews = reviews.splitlines(True)[1:] # 첫번째줄 제거(날짜 때문에)
        reviews = ''.join(reviews) # 리스트를 다시 문자열로 변경
       # print(found_helpful)
        #print(title)
        #print(apphub_CardTextContent)
        #print('-------------------------------------------------')
        df = pd.DataFrame([[helpful, title,date,reviews]])
        if os.path.exists('Dungreed.csv') == False :
            df.columns = ['helpful', 'title','date','reviews']
            df.to_csv('Dungreed.csv', index=False, encoding='utf-8-sig')
        else :
            df.to_csv('Dungreed.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

In [28]:
# 자동
page = 1
code_frame = pd.read_csv('page_list.csv')
name_frame = pd.read_csv('game_list.csv')
name_frame = name_frame.replace(' / ', '/') 
code_list = code_frame['code'].tolist()
name_list = name_frame['title'].tolist()
page_list = code_frame['page'].tolist()
num = 0
page_num = 0
for code in code_list:
    site = 'https://steamcommunity.com/app/%s/homecontent/?' %code
    page_list[page_num]
    for idx in range(round(page_list[page_num] / 10)) :
        time.sleep(2)
        page = idx + 1
        param = {
                    'userreviewsoffset': 10 * (page - 1),
                    'p': page,
                    'workshopitemspage': page,
                    'readytouseitemspage': page,
                    'mtxitemspage': page,
                    'itemspage': page,
                    'screenshotspage': page,
                    'videospage': page,
                    'artpage': page,
                    'allguidepage': page,
                    'webguidepage': page,
                    'integratedguidepage': page,
                    'discussionspage': page,
                    'numperpage': 10,
                    'browsefilter': 'toprated',
                    'appHubSubSection': 10,
                    'l': 'english',
                    'filterLanguage': 'default',
                    'searchText': '',
                    'forceanon': 1
                }

        response = requests.get(site,params=param)
        bs = BeautifulSoup(response.content, 'html.parser')
        div_list = bs.select('.apphub_CardContentMain')
        for apphub_UserReviewCardContent in div_list :
            helpful = apphub_UserReviewCardContent.select('.found_helpful')[0].text
            helpful = helpful.strip()
            title = apphub_UserReviewCardContent.select('.title')[0].text
            date = apphub_UserReviewCardContent.select('.date_posted')[0].text
            date= date.strip()
            date = date.replace('Posted: ','')
            reviews = apphub_UserReviewCardContent.select('.apphub_CardTextContent')[0].text
            reviews = reviews.strip() # 좌우 공백 제거
            reviews = reviews.replace('Product received for free\r\n', '') # 무료 문자 제거
            reviews = reviews.replace('Early Access Review\r\n', '') # 얼리억세스 문자 제거
            reviews = re.sub('[\r\t]','',reviews) # 특수문자 제거
            reviews = reviews.splitlines(True)[1:] # 첫번째줄 제거(날짜 때문에)
            reviews = ''.join(reviews) # 리스트를 다시 문자열로 변경

            df = pd.DataFrame([[helpful, title,date,reviews]])
            if os.path.exists('reviews\%s.csv' %code) == False : # 데이터 저장
                df.columns = ['helpful', 'title','date','reviews']
                df.to_csv('reviews\%s.csv' %code, index=False, encoding='utf-8-sig')
            else :
                df.to_csv('reviews\%s.csv' %code, index=False, encoding='utf-8-sig', mode='a', header=False)
    num=num+1
    page_num = page_num +1

KeyboardInterrupt: 